# **Proyecto Final: (*Identificación de enfermedades en plantas a partir de imágenes*)**

## **Autores**: 
- Vivian Natalia Gómez Cubillos 
- Kelly Katherine Penaranda Rivera

# **Contenido**
1. [**Abstract**](#id1)
2. [**Introducción**](#id2)
3. [**Estado del arte**](#id3)
4. [**Materiales y Métodos**](#id4)
5. [**Resultados**](#id5)
6. [**Conclusión y discusión**](#id6)
7. [**Bibliografía**](#id7)


## **1. Abstract**<a name="id1"></a>
Cuerpo del abstract

## **2. Introducción**<a name="id2"></a>

Cuerpo de la introducción

## **3. Estado del arte**<a name="id3"></a>

Estado del arte

## **4. Materiales y Métodos**<a name="id4"></a>

In [16]:
import utils.visualization as vis
import requests
from skimage import io
import matplotlib.pyplot as plt
import numpy
from skimage import color

In [9]:
#código?

## **5. Resultados**<a name="id5"></a>
Res

## **6. Conclusión y discusión**<a name="id6"></a>

Por medio de todos los casos de prueba realizados con las imágenes seleccionadas y con los resultados obtenidos en cada uno de ellos, pudimos resaltar una serie de observaciones y conclusiones:

### Observaciones:

Obs

### Conclusiones:

En conclusión, 

![ ](https://raw.githubusercontent.com/VivianGomez/t3_segmentation/master/utils/images/umbralsimpleconjeo.PNG)

## **7. Bibliografía**<a name="id7"></a>
Bibliografía